# Projeto 2 - Ciência dos Dados

Nome: Carlos Andrade Inacio

Nome: Carlos Felipe Borges Mesquita

___
Carregando algumas bibliotecas:

In [8]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [9]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Cafeb\OneDrive\Documentos\CDADOS\cdados_projeto2


Carregando a base de dados

In [13]:
filename = 'AllResults1.csv'

In [15]:
train = pd.read_csv(filename)
train

,PPID,PP,EAL,SEN,HML,Re,Wr,Ma,Att8Est,Att8Act,Att8Diff,EngEst,EngAct,EngDiff,MathsEst,MathsAct,MathsDiff,EbaccEst,EbaccAct,EbaccDiff
0,617,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,1.9,NaN,NaN,1.5,NaN,NaN,2.0,NaN,NaN,0.0,NaN
1,283,1.0,1,0.0,NaN,NaN,NaN,NaN,NaN,4.3,NaN,NaN,3.0,NaN,NaN,5.5,NaN,NaN,3.7,NaN
2,75,0.0,0,0.0,M,NaN,NaN,3,2.9,4.1,1.17,3.5,4.0,0.54,2.9,4.0,1.13,1.9,1.3,-0.57
3,1019,1.0,0,0.0,L,NaN,NaN,3,2.2,3.5,1.26,2.8,4.0,1.18,2.0,3.0,0.98,1.2,2.0,0.84
4,474,0.0,0,0.0,L,3C,3B,N,1.7,2.9,1.21,2.2,3.0,0.76,1.3,3.0,1.65,0.7,2.0,1.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1032,174,1.0,0,0.0,M,4B,4,4B,3.9,5.0,1.08,4.3,6.0,1.67,3.6,4.0,0.36,3.5,4.3,0.82
1033,812,1.0,0,0.0,M,5B,3,4C,4.3,3.1,-1.18,4.7,3.0,-1.71,4.1,3.0,-1.08,4.0,3.0,-0.97
1034,425,1.0,0,1.0,H,5B,4,5C,5.5,5.2,-0.30,5.8,6.0,0.20,5.4,4.0,-1.44,5.4,5.3,-0.05
1035,376,1.0,0,0.0,M,5C,4,4B,4.7,1.7,-3.03,5.1,4.0,-1.13,4.6,3.0,-1.58,4.5,1.0,-3.49


In [20]:
train.drop(columns=['PPID','Att8Est','Att8Act','Att8Diff','EngEst','EngAct','EngDiff','MathsEst','MathsAct','MathsDiff','EbaccEst','EbaccAct','EbaccDiff'])

,PP,EAL,SEN,HML,Re,Wr,Ma
0,1.0,0,0.0,NaN,NaN,NaN,NaN
1,1.0,1,0.0,NaN,NaN,NaN,NaN
2,0.0,0,0.0,M,NaN,NaN,3
3,1.0,0,0.0,L,NaN,NaN,3
4,0.0,0,0.0,L,3C,3B,N
...,...,...,...,...,...,...,...
1032,1.0,0,0.0,M,4B,4,4B
1033,1.0,0,0.0,M,5B,3,4C
1034,1.0,0,1.0,H,5B,4,5C
1035,1.0,0,0.0,M,5C,4,4B
